In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.chdir('..')

In [3]:
import pandas as pd
from datasets import load_dataset

/home/alm/miniforge3/envs/assistant_nlp/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from stat_lm import Tokenizer, StatLM, GenerationConfig

In [5]:
# ds_name_2 = 'IlyaGusev/stihi_ru'

def get_dataset(train_size: int,
                test_size: int,
                ds_name_1: str = 'IlyaGusev/gazeta',
               ): 
    
    train_dataset = load_dataset(ds_name_1, split='train')
    test_dataset = load_dataset(ds_name_1, split='test')

    train_df = pd.DataFrame(train_dataset).iloc[:train_size]
    print(train_df.shape)

    test_df = pd.DataFrame(test_dataset)[:test_size]
    print(test_df.shape)

    train_texts = (train_df['title'] + '\n' + train_df['text']).tolist()
    test_texts = (test_df['title'] + '\n' + test_df['text']).tolist()
    
    return train_texts, test_texts

In [6]:
ds_name_1 = "IlyaGusev/gazeta"
train_size = 20000
# train_size = 5000
test_size = 5000

train_dataset = load_dataset(ds_name_1, split='train')
test_dataset = load_dataset(ds_name_1, split='test')

train_df = pd.DataFrame(train_dataset).iloc[:train_size]
print(train_df.shape)

test_df = pd.DataFrame(test_dataset)[:test_size]
print(test_df.shape)

train_texts = (train_df['title'] + '\n' + train_df['text']).tolist()
test_texts = (test_df['title'] + '\n' + test_df['text']).tolist()

/home/alm/miniforge3/envs/assistant_nlp/lib/python3.9/site-packages/datasets/load.py:1461: FutureWarning: The repository for IlyaGusev/gazeta contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/IlyaGusev/gazeta
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(
/home/alm/miniforge3/envs/assistant_nlp/lib/python3.9/site-packages/datasets/load.py:1461: FutureWarning: The repository for IlyaGusev/gazeta contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/IlyaGusev/gazeta
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major rele

(20000, 5)
(5000, 5)


In [7]:
all_texts = train_texts + test_texts

In [8]:
tokenizer = Tokenizer().build_vocab(all_texts)

In [9]:
Tokenizer()

In [10]:
len(tokenizer.vocab)

379713

In [11]:
text_example = "В России люди любят искать приключения на голову"

In [12]:
tokenizer.encode(text_example)

[9867, 335239, 259101, 101824, 209316, 242721, 174755, 20319, 379710]

In [13]:
tokenizer._tokenize(text_example, append_eos_token=False)

['в', 'россии', 'люди', 'любят', 'искать', 'приключения', 'на', 'голову']

In [14]:
tokenizer.decode(tokenizer.encode(text_example), remove_special_tokens=True)

'в россии люди любят искать приключения на голову'

In [15]:
stat_lm = StatLM(tokenizer, context_size=3, alpha=0.0001)

In [16]:
# "обучаем" модель - считаем статистики
stat_lm.train(train_texts)

0it [00:00, ?it/s]

1it [00:21, 21.88s/it]


In [17]:
config = {
    'temperature': 1.2,
    'max_tokens': 30,
    'sample_top_p': 5,
    'decoding_strategy': 'top-p',
    'gen_decay': 1,
}

generation_config = GenerationConfig(temperature=config['temperature'],
                                     max_tokens=config['max_tokens'],
                                     sample_top_p=config['sample_top_p'],
                                     decoding_strategy=config['decoding_strategy'],
                                     gen_decay=config['gen_decay'],
                                     remove_special_tokens=True)

In [18]:
for text in test_texts[50:55]:
    title = text.split('\n')[0]
    generated = stat_lm.generate_text(title, generation_config)
    print(generated['total_text'])
    print(generated['finish_reason'], '\n')

за крупнейшим взломом twitter стоит 16 - летний подросток , меня поймут президент рфс сергей фурсенко , министр спорта виталий мутко . на данный момент в матче . в частности
max tokens 

ni : российский гермес станет убийцей западных танков лута самовозгорания
end of text 

автор хита i like to move it умер на фоне обвинений в насилии . на данный момент в матче с динамо . другие новости и статистику можно посмотреть на странице
max tokens 

названа опасность постоянно включенного bluetooth
end of text 

лавров сообщил о скорой встрече лукашенко с путиным в москве . в итоге , как в россии , а также кубка и суперкубка страны , подробности матчей и положение команд
max tokens 



In [19]:
for text in train_texts[50:55]:
    title = text.split('\n')[0]
    generated = stat_lm.generate_text(title, generation_config)
    print(generated['total_text'])
    print(generated['finish_reason'], '\n')

машины нет в плане , что в случае с ними . я думаю , что в случае с ними , и в других странах . в итоге , по его
max tokens 

за халтуру ответят на сайте ведомства . в итоге , по словам главы ведомства , в том , что на самом деле , а не только с одной стороны ,
max tokens 

кредиты возьмутся за пенсии минздравсоцразвития планирует предоставить гражданам возможность уйти с последнего места в карьер и , конечно же , как и в итоге , как и в этом случае
max tokens 

отдых сбежал от туристов . в начале года . в итоге и стал причиной смерти стал обширный инфаркт . согласно закону о полиции , что это будет очень сложно .
max tokens 

династический спор лейкерс , в частности , в частности , в частности , с помощью которого можно будет увидеть в нашей стране , в частности , в частности , в
max tokens 



In [20]:
tokenizer.save('tokenizer.pkl')

True

In [21]:
stat_lm.save_stat('stat_lm.pkl')

True